Duygu Analizi Model Egitimi


In [46]:
import numpy as np
import pandas as pd
import json
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

dataframe=pd.read_csv('comment.csv',sep='\t')

target=dataframe['Puan'].values.tolist()
data=dataframe['Yorum'].values.tolist()

cutoff=int(len(data)*0.85)
x_train,x_test=data[:cutoff],data[cutoff:]
y_train,y_test=target[:cutoff],target[cutoff:]

#tokenlestirme
with open('tokenizer.json') as json_dosyasi:
    json_tokenizer=json.load(json_dosyasi)

def tokenlestir(yorumListesi):
    y_yorumlar=[]
    for yorum in yorumListesi:
        y_yorum=[]
        for kelime in yorum.split():
            if(len(y_yorum)<50 and kelime in json_tokenizer):
                y_yorum.append(json_tokenizer[kelime])
        if(len(y_yorum)<50):
            sifirlar=list(np.zeros(50-len(y_yorum),dtype=int))
            y_yorum=sifirlar+y_yorum
            
        y_yorumlar.append(y_yorum)
    return np.array(y_yorumlar,dtype=np.dtype(np.int32))





In [43]:
egitim_kume=tokenlestir(x_train)
test_kume=tokenlestir(x_test)
model = Sequential()
model.add(Embedding(input_dim=10001, output_dim=50, input_length=50, name='embedding_layer'))

model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=1e-3)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

model.fit(egitim_kume, np.array(y_train), epochs=15, batch_size=256)
model.evaluate(test_kume, np.array(y_test))
model.save('28Subat2024.h5')

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding  (None, 50, 50)            500050    
 )                                                               
                                                                 
 gru_24 (GRU)                (None, 50, 16)            3264      
                                                                 
 gru_25 (GRU)                (None, 50, 8)             624       
                                                                 
 gru_26 (GRU)                (None, 4)                 168       
                                                                 
 dense_8 (Dense)             (None, 1)                 5         
                                                                 
Total params: 504111 (1.92 MB)
Trainable params: 504111 (1.92 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

/home/user/micromamba/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "gitar kursuna yeni başladım. ne almalıyım diye bakınırken Hepsiburada da bu ürünü gördüm. fiyatı ve hediyeleri ile birlikte çok iyi fiyat. siyah olmasıda benim için ayrı güzel."
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
yazilar = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]
deneme_kume = tokenlestir(yazilar)

deneme_kume[9]
sonuc=model.predict(deneme_kume)
sonuc

1/1 [==============================] - 1s 1s/step


array([[0.99616694],
       [0.9960494 ],
       [0.01889264],
       [0.9961989 ],
       [0.33697012],
       [0.5479653 ],
       [0.97562337],
       [0.874328  ],
       [0.9961584 ],
       [0.03713784]], dtype=float32)